In [1]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import sproc
import snowflake.snowpark.functions as F
from snowflake.snowpark.types import Variant
import json

# 0. Getting Started

Connect to Snowflake, in this case I'm using a creds.json file, it should look like the json I've included in the folder

Note I've commented out the CREATE OR REPLACE DATABASE and STAGE bits and used USE instead, as once I've dragged and dropped the .py file in it wasn't necessary to recreate them every time of course.

In [2]:
snowflake_connection_cfg = json.loads(open("/Users/mitaylor/Documents/creds/creds.json").read())
session = Session.builder.configs(snowflake_connection_cfg).create()
session.sql("CREATE OR REPLACE WAREHOUSE EQT_VWH WITH WAREHOUSE_SIZE='X-SMALL'").collect()
#session.sql("CREATE OR REPLACE DATABASE EQT_DB").collect()
#session.sql("CREATE OR REPLACE STAGE PACKAGE_STAGE").collect()
session.sql("USE DATABASE EQT_DB").collect()

[Row(status='Statement executed successfully.')]

# 1. Pull From a Python File

I thought that this should work, and while I've done it in a slightly different way, this does work, so option 1. as follows:

First, follow these steps to get access to that file:

1. Log in to the Snowflake GUI
2. Look on the left hand side, click Data, then Databases in the drop down, then click on your DB (in this example EQT_DB) in the sheet on the right
3. Then Click on PUBLIC, then STAGES, then PACKAGE STAGE (in this example)
4. Then + Files at the top right, it should be a drag and drop from there

*Note there is an automated way to do all this with SnowCLI too which is probably a better bet long term, esp. given DBT, but this is a good way to get you up and running*

5. Finally, run the SPROC below, which is a python function, that is then registered as a sproc, then called right at the bottom

In [3]:
# Define sproc to test that the import was successful
def py_file_sproc(session: Session) -> Variant:
    import hello_world_file
    return hello_world_file.test_hello_world()

# Register sproc
py_file_sproc = session.sproc.register(
                              func=py_file_sproc, 
                              name='PY_FILE_SPROC', 
                              is_permanent=True, 
                              replace=True,
                              stage_location='@PACKAGE_STAGE', 
                              packages=['snowflake-snowpark-python'],
                              imports=["@PACKAGE_STAGE/hello_world_file.py"])
# Call sproc
py_file_sproc()

'"hello world"'

# 2. Pull From "Local" Machine

Next, the quick and dirty especially for those scenarios where something isn't in our conda channel (https://repo.anaconda.com/pkgs/snowflake/), but is out there in the wild.  Note this only works for pure python packages (so things with weird and wonderful API calls to C++/FORTRAN/SOMETHING_ELSE_ANCIENT libraries etc. won't work.  They can but they need extra dev ourside, so if you need it, lobby us)

Basically, just run the cells below

In [8]:
# just to show it works the same local as in Snowflake
import hello_world_pkg
hello_world_pkg.hello_world.test_hello_world_pkg()

'hello world'

In [9]:
# Get your package path
import hello_world_pkg
hello_world_pkg_path = hello_world_pkg.__path__[0]

In [11]:
# Define sproc to test that the import was successful
def py_local_pkg_sproc(session: Session) -> Variant:
    import hello_world_pkg
    return(hello_world_pkg.hello_world.test_hello_world_pkg())

# Register sproc
py_local_pkg_sproc = session.sproc.register(
                              func=py_local_pkg_sproc, 
                              name='PY_LOC_PKG_SPROC', 
                              is_permanent=True, 
                              replace=True,
                              stage_location='@PACKAGE_STAGE', 
                              packages=['snowflake-snowpark-python'], 
                              imports=[hello_world_pkg_path])
# Call sproc
py_local_pkg_sproc()

'"hello world"'

# 3. Using a .WHL File 

Most packages you find online should be well created, but in the event they aren't create a setup file like I have in this root directory and you can create a .whl file yourself (or you can go to PyPi etc. and grab it if it's there

Navigate to the folder with setup.py in, then run this from the command line

python3 setup.py bdist_wheel

This will create a .whl file inside a dist folder (should be called "hello_world_pkg-1.0-py3-none-any.whl").  Just like in part 1. copy that into your stage, together with the "wheel_loader.py" file.

In [13]:
# Define sproc to test that the import was successful
def py_whl_pkg_sproc(session: Session) -> Variant:
    import wheel_loader
    wheel_loader.load('hello_world_pkg-1.0-py3-none-any.whl')
    import hello_world_pkg 
    return(hello_world_pkg.hello_world.test_hello_world_pkg())

# Register sproc
py_whl_pkg_sproc = session.sproc.register(
                              func=py_whl_pkg_sproc, 
                              name='PY_WHL_PKG_SPROC', 
                              is_permanent=True, 
                              replace=True,
                              stage_location='@PACKAGE_STAGE', 
                              packages=['snowflake-snowpark-python'],
                              imports=["@PACKAGE_STAGE/wheel_loader.py",
                                       "@PACKAGE_STAGE/hello_world_pkg-1.0-py3-none-any.whl"])
# Call sproc
py_whl_pkg_sproc()

'"hello world"'